In [ ]:
using CSV
using DataFrames
using LaTeXStrings
using Plots; pgfplotsx()
using Statistics

In [ ]:
# un-comment and run this cell in order to import the raw data as downloaded from the cluster and aggregate in a single dataframe `df`
# df = DataFrame()
# for i in 1:100 # loop over job indices as set in the .sh file
#     append!(df, DataFrame(CSV.File("csv/input_firing_rate" * lpad(i, 3, "0") * ".csv")))
# end
# df

In [ ]:
# uncomment and run this cell in order to import the aggregated data
df = CSV.read("csv/input_firing_rate.csv", DataFrame)

In [ ]:
# uncomment and run this cell in order to save the aggregated data
# CSV.write("csv/input_firing_rate.csv", df)

In [ ]:
# process data
df2 = groupby(df, [:filter, :rho])
df3 = combine(df2, :mse => mean)
df4 = filter(row -> !isnan(row.mse_mean) && row.mse_mean <= 1, df3)

In [ ]:
# uncomment and run this cell in order to save the aggregated data
CSV.write("csv/input_firing_rate_processed.csv", df4)

In [ ]:
ρ_rng = [2^i for i in 0:8]

In [ ]:
data = filter(row -> row.filter == "DiagSF", df4)
sort!(data, order(:rho))
p = plot(
    data[:, :rho], data[:, :mse_mean], 
    label       = "Diagonal SF", 
    color       = "black",    
    markershape = :circle,
    legend      = :topright,
    xscale      = :log2,
    xticks      = ρ_rng,
    xformatter  = x->Int(x),
    ylims       = (0, 1),
    yticks      = 0:0.2:1,
    xlabel      = "Input firing rate [Hz]",
    ylabel      = "Mean-squared error",
    title       = "Performance as a function of input firing rate",
    titlefont   = (18,),
    tickfont    = (16,),
    guidefont   = (18,),
    legendfontsize = 14
)
data = filter(row -> row.filter == "BlockSF", df4)
sort!(data, order(:rho))
plot!(
    data[:, :rho], data[:, :mse_mean], 
    label             = "Block SF", 
    color             = "blue", 
    markerstrokecolor = "blue",
    markershape       = :circle
)
data = filter(row -> row.filter == "FullSF", df4)
sort!(data, order(:rho))
plot!(
    data[:, :rho], data[:, :mse_mean], 
    label             = "Full SF", 
    color             = "red",
    markerstrokecolor = "red",
    markershape       = :circle
)

In [ ]:
savefig("fig/input_firing_rate.pdf")